In [1]:
#include <iostream>
#include <vector>

#include "xtensor/xarray.hpp"
#include "xtensor/xstridedview.hpp"


template <class D>
void recursive_to_json(xeus::xjson& j, const xt::xexpression<D>& e, xt::slice_vector& slices)
{
    const auto view = xt::dynamic_view(e.derived_cast(), slices);
    if (view.dimension() == 0)
    {
        j = view();
    }
    else
    {
        j = xeus::xjson::array();
        using size_type = typename D::size_type;
        size_type nrows = view.shape()[0];
        for (size_type i = 0; i != nrows; ++i)
        {
            slices.push_back(i);
            xeus::xjson k;
            recursive_to_json(k, e, slices);
            j.push_back(std::move(k));
            slices.pop_back();
        }
    }
}

template <class D>
void to_json(xeus::xjson& j, const xt::xexpression<D>& e)
{
    auto sv = xt::slice_vector(e);
    recursive_to_json(j, e, sv);
}

inline unsigned int json_dimension(const xeus::xjson& j)
{
    if (j.is_array() && j.size())
    {
        return 1 + json_dimension(j[0]);
    }
    else
    {
        return 0;
    }
}

template <class S>
inline void json_shape(const xeus::xjson& j, S& s)
{
    if (j.is_array())
    {
        auto size = j.size();
        s.push_back(size);
        if (size)
        {
            json_shape(j[0], s);
        }
    }
}

In [2]:
xt::xarray<double> t = {{{1, 2}, {3, 4}}, {{1, 2}, {3, 4}}};
xeus::xjson jl;
to_json(jl, t);
std::cout << jl.dump() << std::endl;

[[[1.0,2.0],[3.0,4.0]],[[1.0,2.0],[3.0,4.0]]]


In [3]:
std::cout << json_dimension(jl);

3

In [4]:
std::vector<unsigned int> s;
json_shape(jl, s);

In [5]:
s

(std::vector<unsigned int> &) { 2, 2, 2 }


In [6]:
#include "xtensor/xio.hpp"

In [7]:
template <class D>
inline void from_json_recursive(const xeus::xjson& j, xt::xexpression<D>& e, xt::slice_vector& slices)
{
    auto view = xt::dynamic_view(e.derived_cast(), slices);

    if (view.dimension() == 0)
    {
        view() = j;
    }
    else
    {
        using size_type = typename D::size_type;
        size_type nrows = view.shape()[0];
        for (auto i = 0; i != nrows; ++i)
        {
            slices.push_back(i);
            const xeus::xjson& k = j[i];
            from_json_recursive(k, e, slices);
            slices.pop_back();
        }
    }
}

template <class D>
inline void from_json(const xeus::xjson& j, xt::xexpression<D>& e)
{
    xt::xarray<double> tmp;
    xt::xarray<double>::shape_type s;
    json_shape(j, s);
    tmp.reshape(s);
    auto sv = xt::slice_vector(tmp);
    from_json_recursive(j, tmp, sv);
    e.derived_cast() = std::move(tmp);
}

In [8]:
xt::xarray<double> aa;

In [9]:
from_json(jl, aa);

In [10]:
std::cout << aa << std::endl;

{{{ 1.,  2.},
  { 3.,  4.}},
 {{ 1.,  2.},
  { 3.,  4.}}}
